In [11]:
import mysql.connector
from mysql.connector import errorcode

# 数据库配置
config = {
    'user': 'yangzhilin',
    'password': '4tY93A7Gr4hPU8zb',
    'host': '192.168.137.6',
    'port': 3306,
    'database': 'trade_yangzhilin',
    'raise_on_warnings': True
}

try:
    # 建立数据库连接
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()

    # 确认当前使用的数据库
    cursor.execute("SELECT DATABASE();")
    current_db = cursor.fetchone()[0]
    print(f"当前连接的数据库: {current_db}")

    # 检查表是否存在
    cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables 
        WHERE table_schema = %s 
        AND table_name = 'backtest_result';
    """, (current_db,))
    if cursor.fetchone()[0] == 1:
        print("表 'backtest_result' 已存在。")
    else:
        # 创建表的 SQL 语句
        create_table_query = """
        CREATE TABLE IF NOT EXISTS backtest_result (
            datetime DATETIME NOT NULL,
            name VARCHAR(255) NOT NULL,
            pre_lag INT,
            bar_lag INT,
            frequency VARCHAR(50),
            start_date DATE,
            end_date DATE,
            pot FLOAT,
            hd FLOAT,
            mdd FLOAT,
            wratio FLOAT,
            ir FLOAT,
            ypnl FLOAT,
            sharpe FLOAT,
            max_leverage_ratio FLOAT,
            PRIMARY KEY (datetime, name)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
        """

        # 执行创建表的语句
        cursor.execute(create_table_query)
        print("表 'backtest_result' 创建成功。")

    # 可选：准备插入的数据
    data_to_insert = [
        (
            '2025-01-08 13:56:38', 'xy', 6, 5, '4h', '2025-01-01', '2025-01-07',
            2.2375873202813485, 0.036307396342822, 5.038274066215406,
            0.4523809523809524, -0.15906980228601353, 3.8074875375767157,
            2.275038251613003, 2.718676547310835
        ),
        (
            '2025-01-08 14:58:04', 'xy', 6, 5, '4h', '2025-01-01', '2025-01-07',
            2.2375873202813485, 0.036307396342822, 5.038274066215406,
            0.4523809523809524, -0.15906980228601353, 3.8074875375767157,
            2.275038251613003, 2.718676547310835
        ),
        (
            '2025-01-08 15:07:04', 'xy', 6, 5, '4h', '2025-01-01', '2025-01-07',
            2.2375873202813485, 0.036307396342822, 5.038274066215406,
            0.4523809523809524, -0.15906980228601353, 3.8074875375767157,
            2.275038251613003, None
        )
    ]

    # 创建插入数据的 SQL 语句，使用 ON DUPLICATE KEY UPDATE 来处理重复键
    insert_data_query = """
    INSERT INTO backtest_result (
        datetime, name, pre_lag, bar_lag, frequency, start_date, end_date, 
        pot, hd, mdd, wratio, ir, ypnl, sharpe, max_leverage_ratio
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        pre_lag = VALUES(pre_lag),
        bar_lag = VALUES(bar_lag),
        frequency = VALUES(frequency),
        start_date = VALUES(start_date),
        end_date = VALUES(end_date),
        pot = VALUES(pot),
        hd = VALUES(hd),
        mdd = VALUES(mdd),
        wratio = VALUES(wratio),
        ir = VALUES(ir),
        ypnl = VALUES(ypnl),
        sharpe = VALUES(sharpe),
        max_leverage_ratio = VALUES(max_leverage_ratio);
    """

    # 执行批量插入数据
    cursor.executemany(insert_data_query, data_to_insert)
    cnx.commit()
    print("示例数据已成功插入或更新。")

except mysql.connector.Error as err:
    # 处理不同类型的错误
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("错误: 用户名或密码错误。")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("错误: 数据库不存在。")
    elif err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("错误: 表已存在。")
    else:
        print(f"发生错误: {err}")
finally:
    # 关闭游标和连接
    if 'cursor' in locals():
        cursor.close()
    if 'cnx' in locals() and cnx.is_connected():
        cnx.close()


当前连接的数据库: trade_yangzhilin
表 'backtest_result' 已存在。
示例数据已成功插入或更新。


In [7]:
import time
current_date_str = time.strftime('%Y-%m-%d', time.localtime(time.time()))
current_date_str

'2025-01-10'